# Code snippet for scraping books from rokomari

In [ ]:
from bs4 import BeautifulSoup
import requests
from csv import writer
import pandas as pd

In [ ]:
home_url = "https://www.rokomari.com"
base_url = "https://www.rokomari.com/book/category/1983/extra-discount?ref=act_pg0_p0&page="

title = []
authors = []
prices_original = []
prices_selling = []
discount = []
category = []
publishers = []
ratings_count = []
ratings = []

char_to_replace = {
    '\n': ''
}

for i in range(50):
    url = base_url + str(i+1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    lists = soup.find_all('div', class_="home-details-btn-wrapper")
    
    
    for list in lists:
        for link in list.find_all('a', href=True):
            url_details = home_url + link["href"]
            page_details = requests.get(url_details)
            soup_details = BeautifulSoup(page_details.content, "lxml")
            
            lists_details = soup_details.find_all('div', class_="col details-book-main-info align-self-center")
            lists_authors = soup_details.find_all('p', class_="details-book-info__content-author")
            lists_cat = soup_details.find_all('div', class_="details-book-info__content-category d-flex align-items-center")
            lists_ratings_count = soup_details.find_all('div', 'span', class_="details-book-info__content-rating")
            lists_ratings = soup_details.find_all('div', class_="media ratings-review__content--rating")
            lists_price = soup_details.find_all('div', 'span', class_="details-book-info__content-book-price")
            
            lists_pub = soup_details.find_all('td', class_="publisher-link")    
            
# # # __________________________________Scraping the title of the books_____________________________
            for list_det in lists_details:
                for details in list_det.find_all('div', class_="details-book-main-info__header"):
                    title.append(details.find('h1').text)
                    
#___________________________________Scraping the name of the authors______________________________
            for author in lists_authors:
                authors.append(author.a.text.translate(str.maketrans(char_to_replace)).strip())

# __________________________________Scraping the category______________________
            for cat in lists_cat:
                category.append(cat.a.text.strip())
#___________________________________Scraping the ratings count________________________
            for rating_count in lists_ratings_count:
                rating_count_info = rating_count.span
                if rating_count_info == None:
                    ratings_count.append('NA')
                else:
                    ratings_count.append(rating_count_info.text)
                    
#___________________________________Scraping the ratings (this section has bug)________________________
            for rating in lists_ratings:
                rating_info = rating.find('h2', class_="pt-2")
                if rating_info == None:
                    ratings.append('NA')
                else:
                    ratings.append(rating_info.text.replace('\n', '').strip())

# # ___________________________________scraping the prices_____________________

            for price in lists_price:
                prices_selling.append(price.find('span', class_="sell-price").text.replace('\n', ''))
                price_origin_info = price.find('strike', class_="original-price")
                if price_origin_info == None:
                    prices_original.append('NA')
                    discount.append('Na')
                else:
                    prices_original.append(price_origin_info.text.replace('\n', ''))
                    discount.append(price.find('span', class_="js--save-message").text.replace('\n', ''))

#____________________________________Scraping the publications_________________
                                
            for pub in lists_pub:
                publishers.append(pub.a.string.replace('\n','').strip())
                
        

#****************************  Rearranging all the scraped data ******************************
                
dict_book = {
    "Title": title,
    "Author": authors,
    "Original Price": prices_original,
    "Selling Price": prices_selling,
    "Discount": discount,
    "Publisher": publishers,
    "Rating Count": ratings_count,
    "Category": category
}
book_df = pd.DataFrame(dict_book)   
book_df.to_csv('More Info Rokomari.csv', index=False) # Exporting to a csv file

# Trying To Get the URL

In [ ]:
home_url = "https://www.rokomari.com"
base_path = "https://www.rokomari.com/book/publishers?ref=sm_p2"

publisher = []
url = []

for i in range(3):
    if i<1:
        page = requests.get(base_path)
    else:
        page = requests.get(base_path+'&page='+str(i+1))
    soup = BeautifulSoup(page.content, "lxml")
    lists = soup.find_all('ul', class_="list-inline list-unstyled authorList")
    for list in lists:
        for name in list.find_all('h2'):
            publisher.append(name.text)
        for link in list.find_all('a', href=True):
            url.append(home_url + link["href"]) 

info_dict = {
    "Publishers Name": publisher,
    "URL": url
}
info_df = pd.DataFrame(info_dict)
info_df.to_csv('Publishers_url.csv', index=False)    